## Managing multiple datasets

Data management is one of the most difficult parts of any empirical project. With data readily accessible from surveys, governmental agencies, interest groups, and researchers, today's policy analysts must interact with a wide variety of dataset that they typically do not control the creation of and may have been generated for use-cases quite different from what the analyst has in mind. In this lab, we will learn a number of useful R functions for "data wrangling." While you may never need exactly these tools in your projects, you might need similar tools, so understanding what is possible in R is helpful. 

### The project

Let's examine the relationship between population change and the cost of housing across the U.S., a key public policy topic of interest as land use regulatory reform and rent control policies are being considered at the local and state level across the country. 

We can get population data from the U.S. Census Bureau's Population Estimates program. But that program doesn't have data on cost of housing or rent. The U.S. Department of Housing and Urban Development, however, does have median rents by county. Let's walk through the process of downloading, cleaning, and merging these datasets together. 

First, as always, we load (and download if necessary) required packages. We will be using two primary new packages: **{tigris}**, which provides access to the Census Bureau's geographic data (we'll just use it for access to geographic codes for U.S. counties), and **{rio}**, which provides some nice functions for reading in different types of data. **{rio}** will use the functions available in **{haven}** and in **{readxl}**, so they need to be installed as well. 

In [ ]:
# Install the required packages if not already installed 

# install.packages(c('tidyverse', 'haven', 'tigris', 'readxl', 'rio'))

# let's load your packages in the R session

library(tidyverse)
library(haven)
library(tigris)
library(rio)

Now let's load the population estimates data. We can use the `import()` function from the **{rio}** package. `import()` acts as a wrapper for other functions, like the `read_xlsx` function from **readxl**. This is a neat way to pull in a Microsoft Excel spreadsheet hosted online. 

In [ ]:

# load the url to the data - click on this to view in your web browser
myurl1 <- "https://www2.census.gov/programs-surveys/popest/tables/2020-2022/counties/totals/co-est2022-pop.xlsx"

# the 'col_names', 'sheet' and 'range' functions are part of the `read_xlsx` function, and are 
# passed from import() on to readxl. 
#   col_names - let's us create the variable names we want.
#   sheet - is required with an Excel workbook which can contain multiple sheets
#   range - this is optional, but it lets us select a specific range of cells to
#       avoid headers and footers that aren't part of the data.
popest <- import(myurl1, col_names = c("geo", "popbase", "popest2020", "popest2021", "popest2022"),
                             sheet = "CO-EST2022-POP", 
                             range = "A6:E3149")

head(popest)

Check out the data. It has the county coded as a character variable, with the county name followed by the state. Then we have the population estimates from 2020 through 2022. Another strange aspect of this dataset is that each county has a period (.) at the beginning of the county name. 

In order to merge this dataset with another county-level dataset, we need to separate the county name from the state name and remove the beginning period. We also need to create our population change variables. Let's run `mutate`, which you're familiar with, and include one of R's string functions, `str_remove()`. This function will let us include a pattern of text that we want removed from our variable (variable `geo`). `str_remove()` will remove the first instance of this pattern. In our case, we want the first instance of the period removed.

In [ ]:
popest <- popest |> mutate(change.2020.2021 = popest2021 - popest2020,
                           pct.change.2021 = 100 * change.2020.2021 / popest2020, 
                           change.2020.2022 = popest2022 - popest2020,
                           pct.change.2022 = 100 * change.2020.2022 / popest2020,
                           geo = str_remove(geo, "[.]"))

head(popest)

## For Loops

Now, how should we split county names from state names? We can write a short *loop* to do this. Loops iterate some procedure over a list of objects. Below is very simple loop that grabs each state name (stored in one of R's default datasets and named `state.name`) whenever the stub `st` is called in the code. This loop simply prints the text of the state name. 

In [ ]:
# This is a "for" loop. The beginning of the function describes the iteration (each st in vector state.name)
# inside the curly brackets is the procedure to be repeated 

for(st in state.name){
  print(st)
}

# let's try one more: 

for(i in 1:10){
  print(i^3)
}

Ok, that's nice but not very useful. Let's do something more helpful. With the following loop, let's change the values a state name variable and remove the state name portion from the county variable.

In [ ]:

# first, create an empty variable in the data frame for our state names
popest$state <- NA_character_

# str_ends below identifies if a string ends with some pattern of text 
# that we are storing in the local variable `st`
for(st in state.name){
  ending <- paste(",", st, sep = " ") # will contain the state info like : ", Alabama"
  popest <- popest |> mutate(state = ifelse(str_ends(geo, st), st, state), 
                             geo = str_remove_all(geo, ending)) 
}

head(popest)

And now we have a clean county variable and a separate state name variable!

Let's collect our predictor variable (median rent in the county) from the Department of Housing and Urban Development website using `import()`.

In [ ]:
myurl2 <- "https://www.huduser.gov/portal/datasets/50thper/FY2022_FMR_50_county_rev.xlsx"

# looking at the data frame, the column names from HUD are pretty good. 
# let's use them with col_names = TRUE argument, but replace the spaces
# with periods using the .name_repair argument.
rents <- import(myurl2, col_names = TRUE, .name_repair = make.names,
                sheet = "fy2022_fmr_50_revised")

head(rents)

## Merging data frames

Every modern stats programs have tools to automate the merging of vectors of data. The way these tools generally work is that there is some bridging vector which to identify the units in data frame *x* that will match with units in data frame *y*. R, as you might imagine, has multiple ways of accomplishing this. We'll use the `left_join()` that's part of the **{dplyr}** package in **{tidyverse}**. 

`left_join` takes joins two data frames together. If you don't specify which variable(s) in the `by` argument, then the function will try match based on whatever variables you have in common in both data frames. Don't rely on this. Instead, tell R specifically how to make the match. The other thing to know about `left_join` is that it will keep all units in the data frame you list first, and then include whichever observations are successfully matched in data frame *y*.

Before we can merge `popest` and `rents`, let's take a look at our current geographic identifiers. We've created a county name variable and a state name variable in `popest` - those two variables identify each observation.  `rents`, however, has numeric codes for county and state (called *fips* codes). The easiest way to complete this merge is to first merge `popest` with a separate data frame that includes state and county fips codes, and then merge this combined data frame with the identifying variables with the `rents` data frame. We can pull the data frame `fips_codes` from the **{tigris}** package to access those geographic identifiers.

In [ ]:
# load the fips_codes data frame
data(fips_codes)

# syntax: newdf <- left_join(x, y, by = join_by(ids))

# merge popest and fips_codes by county and state names. the variables
# in popest are listed first in join_by()
popest <- left_join(popest, fips_codes, by = join_by(geo == county, 
                                                     state == state_name))

# let's check what didn't merge (data in popest that wasn't included in fips_codes)
unmatched.obs <- popest |> filter(is.na(state.y))
# if this were a real analysis, we'd want to figure out why each of these isn't
# matching before moving on. But for now, let's skip. 
View(unmatched.obs)

Now that we have our geographic identifiers in `popest`, we should be able to join that data frame with `rents`.

In [ ]:
# in this left_join(), we have the same variable names, state_code and county_code.
# We can use those names with the "==" syntax from before because they're included
# in both data frames. We still need to merge based on value in both state and 
# county, so we must list both.

combined.data <- left_join(rents, popest, by = join_by(state_code, county_code))

We can now create a scatterplot and examine the relationship between median rents and popultion change by county:

In [ ]:
plot <- ggplot(combined.data, aes(x = rent_50_1, y = pct.change.2022)) +
    geom_point(alpha = .1) +
    theme_minimal()
plot

Hmmm. Looks good! But what's going on with those extra dark markers?

When we did the merge, the `fips_codes` data had some county subunits included in New England, what the Census Bureau calls "county equivalent" units. Because we merged on state and county, R pulled in the population estimates data and merged them with the rent data, some of which is collected at the sub-county level. Again, if we were doing a formal analysis, we would want to aggregate these data up to the county-level. But for now, let's simply ignore those observations. We can filter out those observations directly in the `ggplot()`. Try it by adding `|> filter(county_sub_code == "99999")` after the data frame name in the `ggplot` function above.

In [ ]:
plot <- ggplot(combined.data |> filter(county_sub_code == "99999"), aes(x = rent_50_1, y = pct.change.2022)) +
    geom_point(alpha = .1) +
    theme_minimal()
plot

## Facets 

We can add a little more to this analysis. We might be interested in seeing how this relationship varies by region of the country and/or by the population size of the county. Let's process our data a bit more to be able to do that. First, we can merge in a state-level dataset that contains the Census region. Then, we can recode `popest2020` into an ordinal measure of county size. 

In [ ]:
# the vectors state.abb and state.region are part of 
# a built-in R dataset. We can combine them together using
# tibble() because the rows match. 

states <- tibble(state.abb, state.region)

# now run another left_join to match county data to state data

combined.data <- left_join(combined.data, states, by = join_by(state.y == state.abb))

head(combined.data)

combined.data <- combined.data |> mutate(popcat = case_when(
                                            popest2020<100000 ~ 0,
                                            popest2020>=100000 & popest2020<500000 ~ 1,
                                            popest2020>= 500000 ~ 2))

Think of facets as mini-graphs; you can repeat your by some other category of a variable (or multiple variables). We can simply add the `facet_grid` function to our `ggplot` graph:

In [ ]:
# first, let remove any missing observations from faceting variables
# let's also add a regression line

plot2 <- ggplot(combined.data |> filter(county_sub_code == "99999" & 
                                        !is.na(state.region) &
                                        !is.na(popcat)), 
                aes(x = rent_50_1, y = pct.change.2022)) +
            geom_point(alpha = .1) +
            geom_smooth(method = "lm", se = FALSE, color = "red") + 
            theme_minimal()

# graph with facets by region
plot2 + facet_grid( ~ state.region)

# graph with facets by population category
plot2 + facet_grid( ~ popcat)

# graph with facets by both population and region
plot2 + facet_grid(popcat ~ state.region)